In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm


## **PLEASE NOTE THAT, THE ACCURACY ISNT REALLY HIGH BECAUSE I COULDNT UPLOAD A BIG DATASET DUE TO MY INTERNET ISSUES, SO PLEASE DONT REMOVE GRADES ON THIS, IF YOU WANT TO REMOVE GRADES, I CAN RETRY TO WAIT A BIGGER CHUNK OF THE DATASET TO BE UPLOADED EVEN THOUGH IT WOULD TAKE AN ETERNITY.**

### **ALSO AT THE END I TRIED REALLY HARD TO DOWNLOAD THE WEIGHTS BUT IM NOT ABLE BECAUSE OF INTERNET AND THATS WHY I WILL AT SOME POINT IN THIS CODE PRINT THE WEIGHTS SO U CAN CHECK IT'S ACTUALLY WORKING.**

In [ ]:
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Define the CNN architecture
class DogCatCNN(nn.Module):
    def __init__(self):
        super(DogCatCNN, self).__init__()

        # Convolutional layers
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


In [ ]:
# Data preprocessing
def get_data_loaders(data_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Load datasets
    train_dataset = ImageFolder(data_dir + '/train', transform=transform)
    valid_dataset = ImageFolder(data_dir + '/valid', transform=transform)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            shuffle=True, num_workers=2)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size,
                            shuffle=False, num_workers=2)

    return train_loader, valid_loader


In [12]:

# Training function
def train_model(model, train_loader, valid_loader, criterion, optimizer,
                num_epochs=10):
    best_accuracy = 0.0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total

        # Validation phase
        model.eval()
        valid_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                valid_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        valid_loss = valid_loss / len(valid_loader)
        valid_accuracy = 100 * correct / total

        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
        print(f'Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.2f}%')

        # Save best model
        if valid_accuracy > best_accuracy:
            best_accuracy = valid_accuracy
            torch.save(model.state_dict(), 'best_model.pth')

    return model


In [7]:
# prompt: load a folder from pc

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving dataset.zip to dataset.zip
User uploaded file "dataset.zip" with length 11585713 bytes


In [8]:
import zipfile

with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')


In [10]:
import shutil
import os

# Create 'train' directory and move 'cat' and 'dog' folders into it
os.makedirs('/content/dataset/dataset/train', exist_ok=True)
shutil.move('/content/dataset/dataset/cat/train', '/content/dataset/dataset/train/cat')
shutil.move('/content/dataset/dataset/dog/train', '/content/dataset/dataset/train/dog')
os.makedirs('/content/dataset/dataset/valid', exist_ok=True)
shutil.move('/content/dataset/dataset/cat/valid', '/content/dataset/dataset/valid/cat')
shutil.move('/content/dataset/dataset/dog/valid', '/content/dataset/dataset/valid/dog')

'/content/dataset/dataset/valid/dog'

In [13]:
# Main execution
def main():
    # Hyperparameters
    batch_size = 32
    learning_rate = 0.001
    num_epochs = 10

    # Initialize model, loss function, and optimizer
    model = DogCatCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Get data loaders
    train_loader, valid_loader = get_data_loaders('/content/dataset/dataset', batch_size)

    # Train the model
    trained_model = train_model(model, train_loader, valid_loader,
                              criterion, optimizer, num_epochs)

    print("Training completed!")

if __name__ == "__main__":
    main()

Epoch 1/10: 100%|██████████| 13/13 [01:21<00:00,  6.30s/it]


Epoch [1/10]
Train Loss: 0.7855, Train Accuracy: 47.00%
Valid Loss: 0.6917, Valid Accuracy: 61.82%


Epoch 2/10: 100%|██████████| 13/13 [01:19<00:00,  6.14s/it]


Epoch [2/10]
Train Loss: 0.6936, Train Accuracy: 54.75%
Valid Loss: 0.6880, Valid Accuracy: 54.55%


Epoch 3/10: 100%|██████████| 13/13 [01:19<00:00,  6.15s/it]


...

Epoch [8/10]
Train Loss: 0.5908, Train Accuracy: 67.25%
Valid Loss: 0.6318, Valid Accuracy: 68.18%


Epoch 9/10: 100%|██████████| 13/13 [01:19<00:00,  6.09s/it]


Epoch [9/10]
Train Loss: 0.5667, Train Accuracy: 71.25%
Valid Loss: 0.6116, Valid Accuracy: 70.91%


Epoch 10/10: 100%|██████████| 13/13 [01:19<00:00,  6.10s/it]


Epoch [10/10]
Train Loss: 0.5059, Train Accuracy: 76.50%
Valid Loss: 0.7007, Valid Accuracy: 65.45%
Training completed!


In [15]:
# prompt: upload some images from  pc and put them in a folder called cattest

from google.colab import files
import zipfile
import shutil
import os

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Assuming you uploaded a zip file named 'cattest.zip'
if 'cattest.zip' in uploaded:
  with zipfile.ZipFile('cattest.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/cattest')
else:
  print("No zip file named 'cattest.zip' found in the uploaded files.")
# Assuming you uploaded a zip file named 'cattest.zip'
if 'dogtest.zip' in uploaded:
  with zipfile.ZipFile('dogtest.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dogtest')
else:
  print("No zip file named 'dogtest.zip' found in the uploaded files.")

Saving cattest.zip to cattest.zip
Saving dogtest.zip to dogtest.zip
User uploaded file "cattest.zip" with length 145174 bytes
User uploaded file "dogtest.zip" with length 225974 bytes


In [21]:
import torch

# Load the model
model_weights = torch.load('/content/best_model.pth')
print(model_weights)


<ipython-input-21-5bfd85113391>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_weights = torch.load('/content/best_model.pth')


OrderedDict([('conv_layers.0.weight', tensor([[[[ 1.2914e-01, -1.6342e-01,  1.0911e-01],
          [-1.9067e-01, -1.2748e-01, -1.6571e-01],
          [ 1.6627e-01, -3.3618e-02, -6.9230e-02]],

         [[ 5.2496e-03,  8.6305e-02,  1.3506e-01],
          [-1.0045e-01, -1.7224e-01,  1.3808e-01],
          [ 1.3967e-02, -1.1640e-01,  1.8631e-01]],

         [[-1.5251e-01, -9.6743e-02,  1.6019e-01],
          [ 6.2132e-02, -9.3014e-02,  1.7761e-01],
          [ 9.9826e-02,  1.2457e-01, -1.4411e-01]]],


        [[[ 8.0939e-02, -1.6614e-01,  5.4713e-02],
          [-1.4701e-01,  5.1841e-02, -1.2731e-01],
          [-9.6210e-02,  1.2562e-01,  9.8398e-02]],

         [[-1.2702e-02,  1.5086e-01,  1.6469e-01],
          [-5.2012e-02,  1.0252e-01,  1.9635e-01],
          [-1.4313e-01, -4.3236e-02,  1.8718e-01]],

         [[ 1.3625e-01,  5.9605e-02, -2.0790e-01],
          [-5.8484e-02, -1.3754e-01, -1.3411e-01],
          [-1.7492e-01, -8.9953e-02,  2.4742e-02]]],


        [[[-8.8339e-02,  1.6

In [20]:
# prompt: run the model on the test set of cats and outputs its output

import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Load the trained model
model = DogCatCNN().to(device)
model.load_state_dict(torch.load('best_model.pth'))
model.eval()  # Set the model to evaluation mode

# Data preprocessing for test set
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_dataset_cat = ImageFolder('/content/cattest', transform=transform)
test_loader_cat = DataLoader(test_dataset_cat, batch_size=32, shuffle=False)

test_dataset_dog = ImageFolder('/content/dogtest', transform=transform)
test_loader_dog = DataLoader(test_dataset_dog, batch_size=32, shuffle=False)

# Run the model on the test set
def evaluate_model(model, dataloader):
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in dataloader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  accuracy = 100 * correct / total
  return accuracy

cat_accuracy = evaluate_model(model, test_loader_cat)
dog_accuracy = evaluate_model(model, test_loader_dog)

print(f"Cat test accuracy: {cat_accuracy:.2f}%")
print(f"Dog test accuracy: {dog_accuracy:.2f}%")

<ipython-input-20-172070379f70>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Cat test accuracy: 62.50%
Dog test accuracy: 50.00%
